# Querying a GitHub Codebase with Vertex AI RAG Engine

This notebook demonstrates how to use Vertex AI's Retrieval-Augmented Generation (RAG) capabilities to index the code files from a public GitHub repository and then ask questions about that codebase using a generative model.

It uses [Vertex AI RAG Engine](https://cloud.google.com/vertex-ai/generative-ai/docs/rag-engine/rag-overview), a component of the Vertex AI Platform.

## Setup

### Install Packages

This cell installs the required Python libraries:
*   [`google-cloud-aiplatform`](https://cloud.google.com/python/docs/reference/aiplatform/latest): The Vertex AI SDK for interacting with Vertex AI services like RAG and Gemini models.
*   [`google-cloud-storage`](https://cloud.google.com/python/docs/reference/storage/latest): The SDK for interacting with Google Cloud Storage (GCS).
*   [`gitpython`](https://gitpython.readthedocs.io/en/stable/): A library to interact with Git repositories (cloning).
*   [`google-genai`](https://github.com/googleapis/python-genai): The Google Generative AI SDK (used here for simplified client interaction via `genai.Client`).

It also restarts the Colab kernel, which is often necessary after installing new packages.

In [ ]:
%pip install --upgrade --user google-cloud-aiplatform google-cloud-storage gitpython google-genai

# Restart kernel after installs
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

### Authentication

This cell handles authentication, which is necessary to interact with Google Cloud services. If running in [Google Colab](https://colab.research.google.com/), it uses `google.colab.auth` to authenticate the user. If running in a different environment (like a local machine or a Vertex AI Workbench notebook), you might need to set up authentication differently (e.g., using `gcloud auth application-default login`).

In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### Import Libraries

Imports the necessary Python modules used throughout the notebook.

In [ ]:
import os
import uuid

import git
from google import genai
from google.cloud import storage
from google.genai.types import GenerateContentConfig, Retrieval, Tool, VertexRagStore
from IPython.display import display, HTML, Markdown
import vertexai
from vertexai import rag

## Configuration

### Define Parameters

Set the configuration variables required for the notebook. **You must replace placeholder values (like `[your-project-id]`, `[your-bucket-name]`) with your specific details.**

*   `GITHUB_URL`: The URL of the public GitHub repository to index.
*   `PROJECT_ID`: Your Google Cloud Project ID.
*   `LOCATION`: The Google Cloud region where resources will be created (e.g., `us-central1`).
*   `BUCKET_NAME`: The name of the GCS bucket used for staging code files. Ensure this bucket exists in your project.
*   `GCS_FOLDER_PATH`: An optional prefix (folder path) within the bucket to organize the uploaded code files.
*   `MAX_FILE_SIZE_MB`: A limit to skip uploading very large files (0 means no limit).
*   `EMBEDDING_MODEL`: The [Vertex AI text embedding model](https://cloud.google.com/vertex-ai/generative-ai/docs/embeddings/get-text-embeddings) used to create vector representations of the code chunks.
*   `MODEL_ID`: The [Vertex AI Gemini model](https://cloud.google.com/vertex-ai/generative-ai/docs/models) used for answering questions.
*   `RAG_CORPUS_DISPLAY_NAME`, `RAG_ENGINE_DISPLAY_NAME`: Unique display names for the RAG resources being created.
*   `SUPPORTED_EXTENSIONS`: A list of file extensions and specific filenames to identify code, configuration, and documentation files for indexing.

In [ ]:
GITHUB_URL = "https://github.com/google/adk-python"  #@param {"type":"string", "placeholder": "https://github.com/google/adk-python"}
PROJECT_ID = "your-project-id"  #@param {type:"string", placeholder: "[your-project-id]", isTemplate: true}
LOCATION = "us-central1"              #@param {type:"string"}
BUCKET_NAME = "your-bucket-name"  #@param {type:"string", placeholder: "[your-bucket-name]", isTemplate: true}
GCS_FOLDER_PATH = "rag-code-data" #@param {type:"string", placeholder: "rag-code-data", isTemplate: true} - The folder path (prefix) within the bucket where code files will be stored.
MAX_FILE_SIZE_MB = 10 #@param {type:"number"} - Maximum individual file size in MB to upload. Set to 0 for no limit.
EMBEDDING_MODEL = "publishers/google/models/text-multilingual-embedding-002"  # @param {type:"string", isTemplate: true}
MODEL_ID = "gemini-2.5-flash"  # @param {type:"string", "placeholder": "gemini-2.5-flash", isTemplate: true}

GCS_FOLDER_PATH = GCS_FOLDER_PATH.strip('/')
BUCKET_URI = f"gs://{BUCKET_NAME}"
GCS_UPLOAD_URI = f"{BUCKET_URI}/{GCS_FOLDER_PATH}" if GCS_FOLDER_PATH else BUCKET_URI
GCS_IMPORT_URI = f"{GCS_UPLOAD_URI}/"
LOCAL_REPO_PATH = "./cloned_repo"

# RAG Engine Configuration
_UUID = uuid.uuid4()
RAG_CORPUS_DISPLAY_NAME = f"rag-corpus-code-{_UUID}"
RAG_ENGINE_DISPLAY_NAME = f"rag-engine-code-{_UUID}"

# Supported file extensions for ingestion (adjust as needed)
# Common code, config, and documentation file types
SUPPORTED_EXTENSIONS = [
    ".py", ".java", ".js", ".ts", ".go", ".c", ".cpp", ".h", ".hpp",
    ".cs", ".rb", ".php", ".swift", ".kt", ".scala",
    ".md", ".txt", ".rst", ".html", ".css", ".scss",
    ".yaml", ".yml", ".json", ".xml", ".proto", "Dockerfile", ".sh",
    ".tf", ".tfvars", ".bicep", ".gradle", "pom.xml", "requirements.txt",
    "package.json", "go.mod", "go.sum", "Cargo.toml"
]

### Initialize API Clients

Initializes the clients needed to interact with Google Cloud services using the specified `PROJECT_ID` and `LOCATION`.
*   [`vertexai.init`](https://cloud.google.com/python/docs/reference/aiplatform/latest/vertexai#vertexai_init): Initializes the Vertex AI SDK.
*   [`genai.Client`](https://github.com/google/generative-ai-python/blob/v0.7.1/google/generativeai/client.py#L103): Creates a client for the Generative AI API (convenience wrapper).
*   [`storage.Client`](https://cloud.google.com/python/docs/reference/storage/latest/google.cloud.storage.client.Client): Creates a client for Google Cloud Storage.

In [ ]:
vertexai.init(project=PROJECT_ID, location=LOCATION)
client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)
storage_client = storage.Client(project=PROJECT_ID)

## Data Preparation

### Clone GitHub Repository

Clones the public Git repository specified by `GITHUB_URL` into a local directory (`./cloned_repo`). It uses the [`gitpython`](https://gitpython.readthedocs.io/en/stable/) library. Error handling is included in case the cloning fails (e.g., repository not found, network issues, directory already exists).

*Note: The local path `./cloned_repo` is hardcoded in the `clone_from` command in this cell.*

In [ ]:
try:
  git.Repo.clone_from(GITHUB_URL, LOCAL_REPO_PATH)
except git.GitCommandError as e:
  print(f"Error cloning repository: {e}")

### Verify GCS Bucket Access

Checks if the specified [Google Cloud Storage bucket](https://cloud.google.com/storage/docs/buckets) (`BUCKET_NAME`) exists and if the authenticated user or service account has permissions to access it. This helps catch configuration errors early.

In [ ]:
try:
    bucket = storage_client.get_bucket(BUCKET_NAME)
    print(bucket.name)
except Exception as e:
    print(f"Error accessing GCS bucket '{BUCKET_NAME}': {e}")

### Upload Code Files to GCS

This cell walks through the locally cloned repository (`LOCAL_REPO_PATH`, implicitly `./cloned_repo` from the clone step) and uploads relevant files to the specified Google Cloud Storage bucket (`BUCKET_NAME`) and folder (`GCS_FOLDER_PATH`).

Key actions:
*   Skips the `.git` directory.
*   Filters files based on `SUPPORTED_EXTENSIONS` (case-insensitive check).
*   Optionally skips files larger than `MAX_FILE_SIZE_MB`.
*   Preserves the relative directory structure within the specified GCS folder.
*   Uploads files as [GCS objects (blobs)](https://cloud.google.com/storage/docs/objects).
*   Prints progress and a final summary.

In [ ]:
uploaded_file_count = 0
skipped_file_count = 0

# Calculate max size in bytes once, if applicable
max_bytes = 0
if MAX_FILE_SIZE_MB > 0:
    max_bytes = MAX_FILE_SIZE_MB * 1024 * 1024
    print(f"Applying max file size limit: {MAX_FILE_SIZE_MB} MB ({max_bytes} bytes)")

for root, dirs, files in os.walk(LOCAL_REPO_PATH):
    # Skip '.git' directory explicitly to avoid uploading git metadata
    if '.git' in dirs:
        dirs.remove('.git')

    for file in files:
        file_lower = file.lower() # Use lowercase for case-insensitive checks
        local_file_path = os.path.join(root, file)

        # Check if the file has a supported extension (case-insensitive)
        # or if the exact filename is in the list (e.g., "Dockerfile")
        is_supported = any(file_lower.endswith(ext.lower()) for ext in SUPPORTED_EXTENSIONS) or \
                       file in SUPPORTED_EXTENSIONS # Check exact match for non-extension files

        if is_supported:
            try:
                if max_bytes > 0: # Only check if a limit is set
                    file_size_bytes = os.path.getsize(local_file_path)
                    if file_size_bytes > max_bytes:
                        print(f"  Skipping large file ({(file_size_bytes / (1024*1024)):.2f} MB > {MAX_FILE_SIZE_MB} MB): {local_file_path}")
                        skipped_file_count += 1
                        continue # Skip to the next file

                # Create a relative path to maintain structure in GCS
                relative_path = os.path.relpath(local_file_path, LOCAL_REPO_PATH)

                # Construct the destination blob name within the specified GCS folder path
                if GCS_FOLDER_PATH:
                    gcs_blob_name = os.path.join(GCS_FOLDER_PATH, relative_path)
                else:
                    gcs_blob_name = relative_path
                gcs_blob_name = gcs_blob_name.replace("\\", "/")

                # Get the blob object and upload the file
                blob = bucket.blob(gcs_blob_name)
                blob.upload_from_filename(local_file_path)
                uploaded_file_count += 1

                # Print progress periodically
                if uploaded_file_count % 100 == 0:
                    print(f"  Uploaded {uploaded_file_count} files (skipped: {skipped_file_count})...")

            except Exception as e:
                print(f"  Error uploading {local_file_path} to gs://{BUCKET_NAME}/{gcs_blob_name}: {e}")

# --- Final Report ---
print(f"\nFinished uploading.")
print(f"Total supported files uploaded: {uploaded_file_count}")
if skipped_file_count > 0:
    print(f"Total files skipped due to size limit: {skipped_file_count}")

if uploaded_file_count == 0:
    print(f"\nWarning: No supported files were found (within size limits) in '{LOCAL_REPO_PATH}' or uploaded to '{GCS_UPLOAD_URI}'.")
    print("The RAG Engine will have no data from this source.")

## 4. RAG Corpus Creation

Creates a [Vertex AI RAG Corpus](https://cloud.google.com/vertex-ai/generative-ai/docs/rag-engine/manage-your-rag-corpus). A RAG Corpus is a container for the documents (in this case, code files) that will be indexed for retrieval.

*   It's configured with a display name and description.
*   It specifies the `EMBEDDING_MODEL` to be used for generating vector embeddings of the file contents during the import process. The embeddings are stored in an underlying vector database managed by Vertex AI.

In [ ]:
rag_corpus = rag.create_corpus(
    display_name=RAG_CORPUS_DISPLAY_NAME,
    description=f"Codebase files from {GITHUB_URL}",
    backend_config=rag.RagVectorDbConfig(
        rag_embedding_model_config=rag.RagEmbeddingModelConfig(
            vertex_prediction_endpoint=rag.VertexPredictionEndpoint(
                publisher_model=EMBEDDING_MODEL
            )
        )
    )
)
rag_corpus.display_name

## 5. File Ingestion: Import Files into Corpus

Starts the asynchronous process of importing files from the GCS location (`GCS_IMPORT_URI`) into the newly created `rag_corpus`.

*   [`rag.import_files`](https://cloud.google.com/vertex-ai/generative-ai/docs/rag-engine/use-data-ingestion): Initiates the import job.
*   **Chunking:** The `TransformationConfig` specifies how the files are processed before indexing. Here, `ChunkingConfig` defines that files are split into chunks of 500 tokens with an overlap of 100 tokens. Chunking is essential for RAG as it allows the model to retrieve smaller, more relevant pieces of information.

This process can take some time depending on the number and size of files.

In [ ]:
import_response = rag.import_files(
    corpus_name=rag_corpus.name,
    paths=[GCS_IMPORT_URI],
    transformation_config=rag.TransformationConfig(
        chunking_config=rag.ChunkingConfig(
            chunk_size=1024,
            chunk_overlap=256
        )
    ),
)

## 6. RAG Tool Setup: Create Retrieval Tool

Creates a `Tool` object that the Gemini model can use to perform retrieval from the RAG Corpus.

*   [`Tool`](https://cloud.google.com/python/docs/reference/aiplatform/latest/vertexai.generative_models.Tool): A generic container for tools that generative models can use (e.g., function calling, retrieval).
*   [`Retrieval`](https://googleapis.github.io/python-genai/genai.html#genai.types.Retrieval): Specifies a retrieval source.
*   [`VertexRagStore`](https://googleapis.github.io/python-genai/genai.html#genai.types.VertexRagStore): Points the retrieval tool specifically to one or more Vertex AI RAG Corpora (`rag_corpora=[rag_corpus.name]`).
  *   `similarity_top_k=10`: Instructs the tool to retrieve the top 10 most relevant chunks based on semantic similarity.
  *   `vector_distance_threshold=0.5`: Sets a threshold for the vector distance; chunks with distances greater than this (less similar) might be filtered out. (Note: The interpretation might vary depending on the distance metric used by the embedding model).

This tool configuration tells the Gemini model *how* to use the RAG Corpus when it needs external information to answer a query.

In [ ]:
rag_retrieval_tool = Tool(
    retrieval=Retrieval(
        vertex_rag_store=VertexRagStore(
            rag_corpora=[rag_corpus.name],
            similarity_top_k=10,
            vector_distance_threshold=0.5,
        )
    )
)

rag_retrieval_tool

## Ask Questions about the Codebase

Now, use the configured Gemini model (`MODEL_ID`) along with the `rag_retrieval_tool` to ask a question about the indexed codebase.

*   The `GenerateContentConfig(tools=[rag_retrieval_tool])` tells the model it can use the RAG tool.
*   When you provide a prompt (like "What is the primary purpose..."), the model first attempts to retrieve relevant chunks from the RAG Corpus using the tool.
*   It then synthesizes an answer based on both its internal knowledge and the retrieved information from the codebase chunks.
*   The response text is displayed as Markdown.

In [ ]:
response = client.models.generate_content(
    model=MODEL_ID,
    contents="What is the primary purpose or main functionality of this codebase?",
    config=GenerateContentConfig(tools=[rag_retrieval_tool]),
)

display(Markdown(response.text))

## 8. Test in Vertex AI Studio

This cell generates a direct clickable link to the [Vertex AI Studio](https://console.cloud.google.com/vertex-ai/studio) in the Google Cloud Console. The link is specifically constructed to open the Multimodal Studio page with the RAG Corpus created in this notebook already selected. This allows you to interactively test the RAG setup by asking questions directly in the UI.

*   It URL-encodes the RAG Corpus resource name for use in the URL query parameters.
*   It uses `IPython.display.HTML` to render a clickable link in the notebook output.

In [ ]:
# URL encode the '/' characters in the resource name by replacing them with '%2F'
encoded_rag_corpus_name = rag_corpus.name.replace("/", "%2F")

# Construct the full URL
vertex_ai_studio_url = (
    f"https://console.cloud.google.com/vertex-ai/studio/multimodal"
    f";ragCorpusName={encoded_rag_corpus_name}"
    f"?project={PROJECT_ID}"
)

# Create the HTML link
link_text = f"{rag_corpus.description}"
link_html = f'<a href="{vertex_ai_studio_url}" target="_blank">{link_text}</a>'

# Display the clickable link in Colab output
print(f"Click to test the RAG Corpus in Vertex AI Studio:")
display(HTML(link_html))